In [10]:
from flask import Flask,make_response,request,render_template
import io
from io import StringIO
import pandas as pd
import numpy as np
import pickle
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

In [11]:
app=Flask(__name__)

In [12]:
def feature_engineering(df):
    df.columns=['age','workclass','fnlwgt','education','education_num','martial_status','occupation','relationship','race','sex','capital_gain','capital_loss','hours_per_week','native_country','salary']
    df=df.drop('fnlwgt',axis=1)
    df['sex']=np.where(df['sex']=="Male",1,0)
    
    label_enco_race={value:key for key,value in enumerate(df['race'].unique())}
    df['race']=df['race'].map(label_enco_race)
    
    label_enco_relation={value:key for key,value in enumerate(df['relationship'].unique())}
    df['relationship']=df['relationship'].map(label_enco_relation)
    
    df['occupation']=np.where(df['occupation']=='?','Missing',df['occupation'])
    label_enco_occu={value:key for key,value in enumerate(df['occupation'].unique())}
    df['occupation']=df['occupation'].map(label_enco_occu)
    
    label_enco_martial_status={value:key for key,value in enumerate(df['martial_status'].unique())}
    df['martial_status']=df['martial_status'].map(label_enco_martial_status)
    
    label_enco_edu={value:key for key,value in enumerate(df['education'].unique())}
    df['education']=df['education'].map(label_enco_edu)
    
    df['workclass']=np.where(df['workclass']=='?','Missing',df['workclass'])
    label_enco_workclass={value:key for key,value in enumerate(df['workclass'].unique())}
    df['workclass']=df['workclass'].map(label_enco_workclass)
    
    df['native_country']=np.where(df['native_country']=='?','Missing',df['native_country'])
    label_enco_native_country={value:key for key,value in enumerate(df['native_country'].unique())}
    df['native_country']=df['native_country'].map(label_enco_native_country)
    return df

In [13]:
def scalar(df):
    sc=StandardScaler()
    X=df[['age','workclass','education','education_num','martial_status','occupation','relationship','race','sex','capital_gain','capital_loss','hours_per_week','native_country','salary']]
    X=sc.fit_transform(X)
    return (X)

In [ ]:
@app.route('/',methods=['GET'])
def Home():
    return render_template('page.html')

@app.route('/predict',methods=["POST"])
def predict():
    f=request.files['data_file']
    if not f:
        return render_template('index.html',prediction_text="No file selected")
    stream=io.StringIO(f.stream.read().decode("UTF8"),newline=None)
    result=stream.read()
    df=pd.read_csv(StringIO(result))
    df=feature_engineering(df)
    X=scalar(df)
    loaded_model=pickle.load(open("lg_model.pkl",'rb'))
    print(loaded_model)
    result=loaded_model.pedict(X)
    return render_template('page.html',predict_text="Predicted Salary is/are:{}".format(result))

if __name__=="__main__":
    app.run(debug=False,port=9000)
    

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:9000
Press CTRL+C to quit
[2023-11-17 14:35:34,393] ERROR in app: Exception on / [GET]
Traceback (most recent call last):
  File "C:\ProgramData\anaconda3\Lib\site-packages\flask\app.py", line 2525, in wsgi_app
    response = self.full_dispatch_request()
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\ProgramData\anaconda3\Lib\site-packages\flask\app.py", line 1822, in full_dispatch_request
    rv = self.handle_user_exception(e)
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\ProgramData\anaconda3\Lib\site-packages\flask\app.py", line 1820, in full_dispatch_request
    rv = self.dispatch_request()
         ^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\ProgramData\anaconda3\Lib\site-packages\flask\app.py", line 1796, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\lenovo\AppData\Local\Temp\ipykernel_2096\4009787159.py

127.0.0.1 - - [17/Nov/2023 14:36:53] "GET / HTTP/1.1" 500 -
